In [15]:
import os
import pdb

import numpy as np
import pandas as pd
from sklearn import preprocessing


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

In [17]:
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

In [18]:
for f in test_identity.columns:
    if f[2] == '-':
        new_name = f[:2] + '_' + f[3:]
        test_identity = test_identity.rename(columns={f: new_name})

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()
del train_transaction, train_identity, test_transaction, test_identity

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

del train, test

X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))


(590540, 433)
(506691, 432)


In [44]:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=432))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               221696    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 291,521
Trainable params: 291,521
Non-trainable params: 0
_________________________________________________________________


In [45]:

# Compiling the model using categorical_crossentropy loss, and rmsprop optimizer.
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [26]:
X_train.iloc[:1000,:].shape

(1000, 432)

In [48]:
X_train_short = X_train.iloc[:5000,:]

In [49]:
y_train_short = y_train.iloc[:5000]

In [50]:
model.fit(X_train_short, y_train_short, epochs=200, batch_size=100, verbose=0)

In [51]:
score = model.evaluate(X_train.iloc[-1000:,:], y_train.iloc[-1000:], verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.04800000041723251
